In [12]:
COCO_ROOT = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/train"

LOG_PATH = "C:/Users/phiho/Projects/GAN_Yolov8/analysis/DogCatPerson.xlsx"
TRAIN_PATH = "C:/Users/phiho/Projects/GAN_Yolov8/Dataset/train_dog_cat_person/299x299"
TEST_PATH = "C:/Users/phiho/Projects/GAN_Yolov8/Dataset/test_dog_cat_person/299x299"

IMG_TRUE_LABLES_PATH = "C:/Users/phiho/Projects/GAN_Yolov8/analysis/COCO_TRAIN_LABELS_PREDICT_TRUE.txt"
IMG_FALSE_LABLES_PATH = "C:/Users/phiho/Projects/GAN_Yolov8/analysis/COCO_TRAIN_LABELS_PREDICT_FALSE.txt"
IMG_PREDICT_LABELS_PATH = "C:/Users/phiho/Projects/GAN_Yolov8/analysis/COCO_TRAIN_LABELS.txt"
IMG_PREDICT_LABELS = {}
IMG_TRUE_LABLES = {}
IMG_FALSE_LABLES = {}

In [47]:
COCO_TRAIN_LABELS = { 'person': [], 'bicycle': [], 'car': [], 'motorcycle': [], 'airplane': [], 'bus': [], 'train': [], 'truck': [], 
               'boat': [], 'traffic light': [], 'fire hydrant': [], 'stop sign': [], 'parking meter': [], 'bench': [], 'bird': [], 
               'cat': [], 'dog': [], 'horse': [], 'sheep': [], 'cow': [], 'elephant': [], 'bear': [], 'zebra': [], 'giraffe': [], 
               'backpack': [], 'umbrella': [], 'handbag': [], 'tie': [], 'suitcase': [], 'frisbee': [], 'skis': [], 'snowboard': [], 
               'sports ball': [], 'kite': [], 'baseball bat': [], 'baseball glove': [], 'skateboard': [], 'surfboard': [], 'tennis racket': [], 
               'bottle': [], 'wine glass': [], 'cup': [], 'fork': [], 'knife': [], 'spoon': [], 'bowl': [], 'banana': [], 'apple': [], 
               'sandwich': [], 'orange': [], 'broccoli': [], 'carrot': [], 'hot dog': [], 'pizza': [], 'donut': [], 'cake': [], 'chair': [], 
               'couch': [], 'potted plant': [], 'bed': [], 'dining table': [], 'toilet': [], 'tv': [], 'laptop': [], 'mouse': [], 'remote': [], 
               'keyboard': [], 'cell phone': [], 'microwave': [], 'oven': [], 'toaster': [], 'sink': [], 'refrigerator': [], 'book': [], 
               'clock': [], 'vase': [], 'scissors': [], 'teddy bear': [], 'hair drier': [], 'toothbrush': [] }
COCO_CLASS_NAMES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
            'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
            'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
            'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
            'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
            'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
            'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
            'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
            'hair drier', 'toothbrush']

COCO_ROOT = "C:/Users/phiho/Projects/Dataset/MS-COCO/COCO_81/coco81/train/"

import os
for i in range(0, len(COCO_CLASS_NAMES)):
    imgs_path_train = COCO_ROOT + COCO_CLASS_NAMES[i]
    for root, dirs, files in os.walk(imgs_path_train):
        for file in files:
            COCO_TRAIN_LABELS[COCO_CLASS_NAMES[i]].append(os.path.join(root, file).split("//")[-1])

# Utils
def get_ground_truth(img_path):
    if '//' in img_path:
        img_path = img_path.split("//")[-1]
    if '/' in img_path:
        img_path = img_path.split("/")[-1]
    if '\\' in img_path:
        img_path = img_path.split("\\")[-1]

    for i in range(0, len(COCO_CLASS_NAMES)):
        result_string = ''.join(COCO_TRAIN_LABELS[COCO_CLASS_NAMES[i]])
        if img_path in result_string:
            return i
    return None    

In [14]:
import json

def load_json_from_txt(path):
    with open(path, 'r') as file:
        data = file.read()
    return json.loads(data)

def count_crop_img(image_names):
    unique_prefixes = set()
    for image_name in image_names:
        prefix = image_name.split('_')[0]
        unique_prefixes.add(prefix)
    return len(unique_prefixes)

IMG_PREDICT_LABELS = load_json_from_txt(IMG_PREDICT_LABELS_PATH)
IMG_FALSE_LABLES = load_json_from_txt(IMG_FALSE_LABLES_PATH)
IMG_TRUE_LABLES = load_json_from_txt(IMG_TRUE_LABLES_PATH)

In [15]:
import os
import random
import shutil
import pandas as pd

CLASS_NAMES = ['dog', 'cat', 'person']
data = []

for label_name in CLASS_NAMES:
    # Count cropped images
    count_crop_all = count_crop_img(os.listdir(os.path.join(COCO_ROOT, label_name)))
    count_crop_process = 0
    count_crop_train = 0
    count_crop_test = 0

    subfolder_path = os.path.join(COCO_ROOT, label_name)
    desfolder_path = os.path.join(TRAIN_PATH, label_name)
    test_path = os.path.join(TEST_PATH, label_name)
    if not os.path.exists(desfolder_path):
        os.makedirs(desfolder_path)
    if not os.path.exists(test_path):
        os.makedirs(test_path)
    
    num_files = min([len(IMG_TRUE_LABLES['dog']), len(IMG_TRUE_LABLES['cat']), len(IMG_TRUE_LABLES['person'])])

    # File true
    files_true = [f for f in IMG_TRUE_LABLES[label_name] if os.path.isfile(os.path.join(subfolder_path, f))]
    num_files_true_to_move = int(num_files * 96 / 100)
    files_true_to_move = random.sample(files_true, num_files_true_to_move)
    
    for file in files_true_to_move:
        src_path = os.path.join(subfolder_path, file)
        dst_path = os.path.join(desfolder_path, file)
        shutil.copy(src_path, dst_path)
        shutil.copy(src_path, os.path.join("C:/Users/phiho/Projects/GAN_Yolov8/Dataset/train_and_test_dog_cat_person/299x299", file))

    # File false
    files_false = [f for f in IMG_FALSE_LABLES[label_name] if os.path.isfile(os.path.join(subfolder_path, f))]
    num_files_false_to_move = num_files - num_files_true_to_move
    files_false_to_move = random.sample(files_false, num_files_false_to_move)
    
    for file in files_false_to_move:
        src_path = os.path.join(subfolder_path, file)
        dst_path = os.path.join(desfolder_path, file)
        shutil.copy(src_path, dst_path)
        shutil.copy(src_path, os.path.join("C:/Users/phiho/Projects/GAN_Yolov8/Dataset/train_and_test_dog_cat_person/299x299", file))

    # Merge both true and false
    files_both = files_true_to_move + files_false_to_move
    count_crop_process = count_crop_img(files_both)

    # File test
    num_files_test_to_move = int(len(files_both) * 0.2)
    files_test_to_move = random.sample(files_both, num_files_test_to_move)
    num_true_test = 0
    num_false_test = 0

    for file in files_test_to_move:
        if file in IMG_TRUE_LABLES[label_name]: num_true_test += 1
        elif file in IMG_FALSE_LABLES[label_name]: num_false_test += 1
        else: 
            print(label_name, file)
            print("Error")

        src_path = os.path.join(desfolder_path, file)
        dst_path = os.path.join(test_path, file)
        shutil.move(src_path, dst_path)
    
    count_crop_test = count_crop_img(files_test_to_move)
    count_crop_train = count_crop_img([x for x in files_both if x not in files_test_to_move])

    # Save to data frame
    data_i = []
    data_i.append(label_name)
    data_i.append(len(IMG_PREDICT_LABELS[label_name]))
    data_i.append(count_crop_all)
    data_i.append(f"{len(IMG_TRUE_LABLES[label_name])/len(IMG_PREDICT_LABELS[label_name])*100:.2f}%")
    data_i.append(num_files)
    data_i.append(count_crop_process)
    data_i.append(f"{num_files_true_to_move/num_files*100}%")
    data_i.append(len(files_both) - num_files_test_to_move)
    data_i.append(count_crop_train)
    data_i.append(f"{(num_files_true_to_move-num_true_test)/(len(files_both)-num_files_test_to_move)*100:.2f}%")
    data_i.append(num_files_test_to_move)
    data_i.append(count_crop_test)
    data_i.append(f"{num_true_test/num_files_test_to_move*100:.2f}%")

    data.append(data_i)

In [16]:
df = pd.DataFrame(data, columns=['Class', 
                                 'Number of all images', 'Number of cropped images', 'Raw accuracy', 
                                 'Number of processed images', 'Number of cropped images', 'Accuracy after processing', 
                                 'Number of training images', 'Number of cropped images', 'Accuracy of training images', 
                                 'Number of testing images', 'Number of cropped images', 'Accuracy of testing images'])
df.to_excel(LOG_PATH, sheet_name='All', index=False)

In [24]:
import numpy as np

def get_top5_idx_highest(arr):
    sorted_indices = np.argsort(arr)
    top_5_indices = sorted_indices[-5:]
    top_5_indices = top_5_indices[::-1]
    return top_5_indices

In [26]:
import os
from yolov8_infer import model
import torch
import torchvision.datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

all_imgs_path = []
folder_path = "C:/Users/phiho/Projects/GAN_Yolov8/Dataset/train_and_test_dog_cat_person/299x299"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    all_imgs_path.append(file_path)
print(len(all_imgs_path))

coco_dataset = torchvision.datasets.ImageFolder('./dataset/train_and_test_dog_cat_person', transform=transforms.ToTensor())
dataloader = DataLoader(coco_dataset, batch_size=len(all_imgs_path), shuffle=False, drop_last=True)
print("All image examples: ", coco_dataset.__len__())


6969
All image examples:  6969


In [19]:
for data in dataloader:
    print(len(data))
images,labels = data
device = torch.device("cuda" if (torch.cuda.is_available()) else "cpu")
images = images.to(device)
matrix_predict = model(images).cpu()

2


In [48]:
log_data = {
    'dog': [
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 }
    ],
    'cat' : [
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 }
    ], 
    'person' : [
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 },
        { 'person': 0, 'bicycle': 0, 'car': 0, 'motorcycle': 0, 'airplane': 0, 'bus': 0, 'train': 0, 'truck': 0, 'boat': 0, 'traffic light': 0, 'fire hydrant': 0, 'stop sign': 0, 'parking meter': 0, 'bench': 0, 'bird': 0, 'cat': 0, 'dog': 0, 'horse': 0, 'sheep': 0, 'cow': 0, 'elephant': 0, 'bear': 0, 'zebra': 0, 'giraffe': 0, 'backpack': 0, 'umbrella': 0, 'handbag': 0, 'tie': 0, 'suitcase': 0, 'frisbee': 0, 'skis': 0, 'snowboard': 0, 'sports ball': 0, 'kite': 0, 'baseball bat': 0, 'baseball glove': 0, 'skateboard': 0, 'surfboard': 0, 'tennis racket': 0, 'bottle': 0, 'wine glass': 0, 'cup': 0, 'fork': 0, 'knife': 0, 'spoon': 0, 'bowl': 0, 'banana': 0, 'apple': 0, 'sandwich': 0, 'orange': 0, 'broccoli': 0, 'carrot': 0, 'hot dog': 0, 'pizza': 0, 'donut': 0, 'cake': 0, 'chair': 0, 'couch': 0, 'potted plant': 0, 'bed': 0, 'dining table': 0, 'toilet': 0, 'tv': 0, 'laptop': 0, 'mouse': 0, 'remote': 0, 'keyboard': 0, 'cell phone': 0, 'microwave': 0, 'oven': 0, 'toaster': 0, 'sink': 0, 'refrigerator': 0, 'book': 0, 'clock': 0, 'vase': 0, 'scissors': 0, 'teddy bear': 0, 'hair drier': 0, 'toothbrush': 0 }
    ]
}

count_ = {
    'dog' : 0,
    'cat' : 0,
    'person' : 0
}

names = { 0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush', 80: 'no detection' }
class_names = ['dog', 'cat', 'person']

for i in range(len(matrix_predict)):
    top5 = get_top5_idx_highest(matrix_predict[i].cpu().numpy())
    label_gt = get_ground_truth(all_imgs_path[i])
    # print(label_gt)
    if names[label_gt] not in class_names:
        print("Data error!")
        break
    else:
        count_[names[label_gt]] += 1
    
    for i in range(0, 5):
        log_data[names[label_gt]][i][names[top5[i]]] += 1

In [49]:
for label in class_names:
    for i in range(0, 5):
        for name in COCO_CLASS_NAMES:
            log_data[label][i][name] = f"{log_data[label][i][name] / count_[label] * 100:.2f}%"

In [50]:
import pandas as pd

for name in class_names:
    df = pd.DataFrame(log_data[name])
    with pd.ExcelWriter(LOG_PATH, engine='openpyxl', mode='a') as writer:
        df.to_excel(writer, sheet_name=name, index=True)